In [16]:
import logging
logger = logging.getLogger()
for handler in logger.handlers:
    handler.level = logging.WARNING
handler = logging.FileHandler(filename='crime.log', mode='w')
handler.level = logging.DEBUG
logger.level = logging.DEBUG
logger.addHandler(handler)

# TODO
* Jitter on initial field?
* 1D Model?
* Positivity

In [17]:
%matplotlib inline
from autograd import numpy as np
import numpy as true_np
np.mgrid = true_np.mgrid
import autograd
import bayesian_pdes as bpdes
import sympy as sp
import matplotlib.pyplot as plt
import sympy_rbf as rbf

In [18]:
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)
from matplotlib import animation
animation.Animation._repr_html_ = anim_to_html

In [19]:
l = 4 # grid spacing
l_eff = float(l) # effective grid spacing (fudge factor)
size = 64
dt = 0.01 # time spacing
ndim = 2
z = 4.

omega = 1./15 # attractiveness decay rate
eta = 0.03 # neighbourhood effect
theta = 0.56 # increase in attractiveness following a burglary
Gamma = 0.019 # burglar generation rate
A_0 = 1./30 # initial attractiveness

gamma = Gamma / l_eff**ndim
epsilon = theta*dt
D = l_eff**ndim / dt
#D = 100
b_bar = epsilon*D*gamma / omega

In [20]:
x1, x2, y1, y2 = sp.symbols('x_1, x_2, y_1, y_2')
t, tbar = sp.symbols('t,tbar')

sA, sA_x1, sA_x2, sA_xx = sp.symbols('A, A_x1, A_x2, A_xx')
sA_bar, sA_x1_bar, sA_x2_bar, sA_xx_bar = sp.symbols('A_bar, A_x1_bar, A_x2_bar, A_xx_bar')
srho, srho_bar = sp.symbols('rho, rho_bar')

def laplacian(k, wrt):
    return sum([k.diff(a,a) for a in wrt])

def op_A(k):
    return k.diff(t) - (eta*D / z * laplacian(k, [x1, x2]) - omega*(k - A_0) + epsilon*D*srho*k)
def op_A_bar(k):
    return k.diff(tbar) - (eta*D / z *laplacian(k, [y1, y2]) - omega*(k - A_0) + epsilon*D*srho_bar*k)

#also want first and second derivs wrt x, y
def A_x1(k): return k.diff(x1)
def A_x2(k): return k.diff(x2)
def A_xx(k): return k.diff(x1, x1) + k.diff(x2, x2)
def A_y1(k): return k.diff(y1)
def A_y2(k): return k.diff(y2)
def A_yy(k): return k.diff(y1, y1) + k.diff(y2, y2)

def op_rho(k):
    return k.diff(t) - (D/z*(laplacian(k, [x1, x2]) \
                           - 2*(\
                                1./sA * (k.diff(x1)*sA_x1 + k.diff(x2)*sA_x2) \
                                + k*sA_xx/sA \
                                - k*(sA_x1**2 + sA_x2**2) / (sA)**2\
                               )\
                          ) - k*sA + gamma)

def op_rho_bar(k):
    return k.diff(tbar) - (D/z*(laplacian(k, [y1, y2]) \
                           - 2*(\
                                1./sA_bar * (k.diff(y1)*sA_x1_bar + k.diff(y2)*sA_x2_bar) \
                                + k*sA_xx_bar / sA_bar \
                                - k*(sA_x1_bar**2 + sA_x2_bar**2) / (sA_bar)**2 \
                               )\
                          ) - k*sA_bar + gamma)

ls_x, ls_t = sp.symbols('ls_x, ls_t')
k = 0.01*sp.exp(-((x1-y1)**2 + (x2-y2)**2) / (2*ls_x**2))*(sp.exp(-(t-tbar)**2 / (2*ls_t**2)))
#periodic_spatial = rbf.PeriodicKern.create([x], [y], ls_x, 1.0)
#k_per = periodic_spatial * sp.exp(-(t-tbar)**2 / (2*ls_t**2))

kern = k

In [21]:
op_system_A = bpdes.operator_compilation.compile_sympy(
    [op_A, A_x1, A_x2, A_xx], 
    [op_A_bar, A_y1, A_y2, A_yy], 
    kern, 
    [[x1, x2, t, srho], [y1, y2, tbar, srho_bar], [ls_x, ls_t]], 
    mode='cython',
#    sympy_function_kwargs={
#        'compile_helpers': [periodic_spatial.get_helper()]
#    }
)

In [22]:
op_system_rho = bpdes.operator_compilation.compile_sympy(
    [op_rho],
    [op_rho_bar],
    kern,
    [[x1, x2, t, sA, sA_x1, sA_x2, sA_xx], [y1, y2, tbar, sA_bar, sA_x1_bar, sA_x2_bar, sA_xx_bar], [ls_x, ls_t]],
    mode='cython',
#    sympy_function_kwargs={
#        'compile_helpers': [periodic_spatial.get_helper()]
#    }
)

In [23]:
fun_args = np.array([1.5*l, 1.5*dt])

In [24]:
# NOT TOTALLY CONVINCED BY THESE IN BERTOZZI!!!
A_t0 = A_0 + b_bar
rho_t0 = b_bar / A_t0

In [25]:
Tend = 10.
times = np.arange(0,Tend+dt,dt)
design_grid = np.arange(0, size+l, l)
design_x, design_y = np.meshgrid(design_grid, design_grid)
design_points = np.column_stack([design_x.ravel(), design_y.ravel()])
n_pts = len(design_points)
ixs = np.arange(design_points.shape[0])
A_ixs = np.column_stack([ixs, ixs+len(ixs), ixs+2*len(ixs)])
Identity = ()

def plot_field(field, levels=5):
    to_plot = field.reshape(design_x.shape)
    plt.contourf(design_x, design_y, to_plot, levels)

In [26]:
def reshape_op_mat(mat, design_points, n_ops):
    ix = 0
    design = []
    for i in range(n_ops):
        design_op = mat[ix:ix+len(design_points),:]
        ix += len(design_points)
        design.append(design_op)
    return np.column_stack(design)

In [27]:
samp_ops = [Identity, A_x1, A_x2, A_xx]
samp_ops_bar = [Identity, A_y1, A_y2, A_yy]

def construct_design(t, predict_A, predict_rho):
    design_t = bpdes.parabolic.augment_with_time(design_points, t)
    design_A_t = np.column_stack([design_t, predict_rho])
    design_rho_t = np.column_stack([design_t, predict_A])

    return design_A_t, design_rho_t

In [28]:
design_t = bpdes.parabolic.augment_with_time(design_points, times[0])

predict_rho = rho_t0*np.ones(n_pts)
predict_A = np.column_stack([A_t0*np.ones(n_pts), np.zeros(n_pts), np.zeros(n_pts), np.zeros(n_pts)])

design_A_t, design_rho_t = construct_design(times[0], predict_A, predict_rho)
design_A_predict, design_rho_predict = construct_design(dt, predict_A, predict_rho)

obs_A_t = [(design_A_t, A_t0*np.ones(n_pts))]
obs_rho_t = [(design_rho_t, rho_t0*np.ones(n_pts))]

A_t = bpdes.collocate([Identity], [Identity], obs_A_t, op_system_A, fun_args)
A_t_derivs = A_t.apply_operators(samp_ops, samp_ops_bar)
rho_t = bpdes.collocate([Identity], [Identity], obs_rho_t, op_system_rho, fun_args)

#samp_A = A_t_derivs.sample(np.row_stack([design_A_t, design_A_predict]))
#samp_rho = rho_t.sample(np.row_stack([design_rho_t, design_rho_predict]))
samp_A = A_t_derivs.sample(design_A_t)
samp_rho = rho_t.sample(design_rho_t)

# so we now have a sample from A, rho as well as the prediction of the next value
samp_A = reshape_op_mat(samp_A, design_A_t, 4)

A_samples = [samp_A]
rho_samples = [samp_rho]

for ix,t in enumerate(times[1:]):
    
    last_design_A = design_A_t
    last_design_rho = design_rho_t
    
    # should be predictions in here, not samples! But something seems to be wrong with the forecast.
    design_A_t, design_rho_t = construct_design(t, samp_A, samp_rho)
    design_A_predict, design_rho_predict = construct_design(t+dt, samp_A, samp_rho)
    
    # first solve for A
    obs_A_t = [(last_design_A, samp_A[:,0][:,None]), (design_A_t, np.zeros((n_pts, 1)))]
    A_t = bpdes.collocate([Identity, op_A], [Identity, op_A_bar], obs_A_t, op_system_A, fun_args)
    A_t_derivs = A_t.apply_operators(samp_ops, samp_ops_bar)
    samp_A_tmp = A_t_derivs.sample(design_A_t)
    samp_A = reshape_op_mat(samp_A_tmp, design_A_t, 4)
    
    # now for rho
    obs_rho_t = [(last_design_rho, samp_rho), (design_rho_t, np.zeros((n_pts, 1)))]
    rho_t = bpdes.collocate([Identity, op_rho], [Identity, op_rho_bar], obs_rho_t, op_system_rho, fun_args)
    samp_rho = rho_t.sample(design_rho_t)
    
    A_samples.append(samp_A)
    rho_samples.append(samp_rho)
    
    # log maxima
    rhomax, rhomin = samp_rho.max(), samp_rho.min()
    amax, amin = samp_A[:,0].max(), samp_A[:,0].min()
    
    print('t={} A \in ({:.4f},{:.4f}), rho\in ({:.4f}, {:.4f})'.format(t, amin, amax, rhomin, rhomax))

t=0.01 A \in (0.1331,0.3189), rho\in (0.7423, 1.0444)
t=0.02 A \in (0.2111,0.4478), rho\in (0.8139, 1.3992)
t=0.03 A \in (0.2862,0.6069), rho\in (0.8408, 1.7395)
t=0.04 A \in (0.3616,0.8444), rho\in (0.8039, 1.8908)
t=0.05 A \in (0.5571,1.1267), rho\in (0.8314, 2.2374)
t=0.06 A \in (0.8599,1.6377), rho\in (0.9132, 2.5510)
t=0.07 A \in (1.3130,2.5334), rho\in (0.9931, 2.9403)
t=0.08 A \in (1.9308,3.8615), rho\in (1.0602, 3.4825)
t=0.09 A \in (2.8454,6.1117), rho\in (1.0872, 4.1555)
t=0.1 A \in (4.2772,9.4540), rho\in (1.0784, 4.7911)
t=0.11 A \in (6.2986,13.4585), rho\in (1.0924, 5.2117)
t=0.12 A \in (7.6261,20.0056), rho\in (0.9955, 5.1117)
t=0.13 A \in (9.0336,32.3576), rho\in (0.8813, 4.4723)
t=0.14 A \in (11.0860,49.9411), rho\in (0.6977, 3.9472)
t=0.15 A \in (13.5953,74.6353), rho\in (0.4768, 3.3868)
t=0.16 A \in (16.9754,109.0323), rho\in (0.3253, 2.5732)
t=0.17 A \in (22.0059,153.9924), rho\in (0.2207, 1.7409)
t=0.18 A \in (30.8042,207.2540), rho\in (0.1392, 1.0875)
t=0.19 A \in 

KeyboardInterrupt: 

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
x = design_points
to_anim_left = [s[:,0] for s in A_samples]
to_anim_right = rho_samples
fig, axes = plt.subplots(2,1, figsize=(5,10), sharex=True)
levels_left = np.linspace(np.column_stack(to_anim_left).min(), np.column_stack(to_anim_left).max(), 10)
levels_right = np.linspace(np.column_stack(to_anim_right).min(), np.column_stack(to_anim_right).max(), 10)
#axes[0].set_xlim(design_points.min(), design_points.max())
#axes[0].set_ylim(np.column_stack(to_anim_left).min(), np.column_stack(to_anim_left).max())

#axes[1].set_xlim(design_points.min(), design_points.max())
#axes[1].set_ylim(np.column_stack(to_anim_right).min(), np.column_stack(to_anim_right).max())

#line_A, = axes[0].plot([], [])
#line_rho, = axes[1].plot([], [])

# initialization function: plot the background of each frame
def init():
    return []

# animation function.  This is called sequentially
def animate(i):
    axes[0].contourf(design_x, design_y, to_anim_left[i].reshape(design_x.shape), levels=levels_left)
    axes[1].contourf(design_x, design_y, to_anim_right[i].reshape(design_x.shape), levels=levels_right)
    return []

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(to_anim_left), interval=20, blit=False)
plt.close()
anim

In [ ]:
plot_field(A_samples[-1][:,0], 50); plt.colorbar()

In [ ]:
levels_left

In [ ]:
design_rho_t.shape, design_rho_predict.shape

In [ ]:
obs_A_t

In [ ]:

test_points = np.linspace(0,1,51)[:,None]

def obs_function(t):
    design = bpdes.parabolic.augment_with_time(design_points, t)
    values = np.zeros((design.shape[0], 1))
    return [(design, values)]

def test_function(t):
    return bpdes.parabolic.augment_with_time(test_points, t)

A_t0 = A_0 + Theta*D*eta/omega

initial_points = bpdes.parabolic.augment_with_time(design_points, 0)
init_A = A_t0 * np.ones((initial_points.shape[0], 1))

ics = [(initial_points, init_A)]

posteriors = bpdes.parabolic.solve_parabolic(op_system_A, [op_A], [op_A_bar], times, obs_function, ics, fun_args)

In [ ]:
p_ix = 4
p = posteriors[p_ix]
pts = test_function(times[p_ix])
mu, cov = p.apply_operator([(), A_x, A_xx], [(), A_y, A_yy])(pts)
for i in range(3):
    plt.plot(pts[:,0], mu[i*len(pts):(i+1)*len(pts)], label='D{}'.format(i))
for item in design_points:
    plt.axvline(item, linestyle='--', c='black')
plt.legend()

In [ ]:
mu = []
cov = []
for t, posterior in zip(times, posteriors):
    test_t = test_function(t)
    mu_t, cov_t = posterior(test_t)
    mu.append(mu_t)

In [ ]:
plt.plot(test_points, mu[4]);

In [ ]:
plt.plot(test_points, np.column_stack(mu));

# Now rho

In [ ]:
Identity = ()
def make_rho_locs(t, A, A_x, A_xx):
    locs = bpdes.parabolic.augment_with_time(design_points, t)
    locs = np.column_stack([locs, A, A_x, A_xx])
    return locs
def make_rho_obs(t, A, A_x, A_xx):
    locations = make_rho_locs(t, A, A_x, A_xx)
    return [(locations, np.zeros((locations.shape[0], 1)))]

rho_t0 = 1./l**2
rho_loc_0 = make_rho_locs(times[0], init_A, np.zeros_like(init_A), np.zeros_like(init_A))
init_rho = rho_t0*np.ones((rho_loc_0.shape[0], 1))
rho_obs_0 = [(rho_loc_0, init_rho)]
rho_posterior_t0 = bpdes.collocation.collocate([Identity], [Identity], rho_obs_0, op_system_rho, fun_args)

In [ ]:
A_posterior_t1 = posteriors[1].apply_operator([Identity, A_x, A_xx], [Identity, A_y, A_yy])
samp = A_posterior_t1.sample(obs_function(times[1])[0][0]).T
A_samp, A_x_samp, A_xx_samp = samp[:len(samp)/3], samp[len(samp)/3:2*len(samp)/3], samp[2*len(samp)/3:]

In [ ]:
all_rho_ops = [[Identity], [op_rho]] + [[op_rho]]
all_rho_ops_bar = [[Identity], [op_rho_bar]]
all_rho_obs = [rho_obs_0, make_rho_obs(times[1], A_samp, A_x_samp, A_xx_samp)]
rho_posterior_t1 = bpdes.parabolic.step_forward(all_rho_ops, all_rho_ops_bar, all_rho_obs, op_system_rho, fun_args, rho_posterior_t0)

In [ ]:
rho_posterior_t1